In [1]:
%load_ext autoreload

%autoreload 3

In [2]:
%reload_ext autoreload

In [9]:
!pip install wget
!python -m wget https://chromedriver.storage.googleapis.com/108.0.5359.71/chromedriver_win32.zip

You should consider upgrading via the 'C:\Users\user\.virtualenvs\google-arts-0o2XoGZ3\Scripts\python.exe -m pip install --upgrade pip' command.



Saved under chromedriver_win32.zip


# Packages

In [3]:
import pyprojroot
from pathlib  import Path
import time
from concurrent.futures import ProcessPoolExecutor
import pandas as pd
from src.scraping.scraping_pictures import ScrapingPictures
from src.scraping.scraping_picture_info import ScrapingPictureInfo


# Paths

In [4]:
root_path =  pyprojroot.here()
path_driver = (root_path / "notebooks"/'chromedriver.exe').relative_to('/')
data_raw_folder = (root_path / "data"/'raw')

# Scraping

In [23]:
page_sections = ["WHITE","PINK","YELLOW","PURPLE","BLUE","TEAL","GREEN","ORANGE","RED","BROWN","BLACK"] 
scroll_down_times = 120

In [24]:
def scrapePictures(color) -> pd.DataFrame:
    scraper = ScrapingPictures(path_driver,color)
    scraper.open()
    scraper.scroll_down(scroll_down_times)
    time.sleep(8)
    df = scraper.scrape()
    return df

In [19]:
def parallelScrapePictures() -> pd.DataFrame:
    with ProcessPoolExecutor() as executor:
        scrape_df_list = executor.map(scrapePictures, page_sections)
        scrape_df = pd.concat(list(scrape_df_list), ignore_index=True)
        return scrape_df

In [26]:
t1 = time.perf_counter()
df = parallelScrapePictures() 
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

Open page section YELLOW
Open page section BLUE
Open page section PINK
Open page section TEAL
Open page section BLACK
Open page section GREEN
Open page section RED
Open page section BROWN
Open page section ORANGE
Open page section PURPLE
Open page section WHITE
Finish scrolling down
Finish scrolling down
Finish scrolling down
Finish scrolling down
Finish scrolling down
Finish scrolling down
Finish scrolling down
Finish scrolling down
Finish scrolling down
Finish scrolling downFinish scrolling down

Finished in 373.1641500099995 seconds


In [27]:
len(df)

16378

In [30]:
df.sample(15)

,image,page,color
9526,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/escrav...,BROWN
6055,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/landsc...,GREEN
6703,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/fenest...,GREEN
10053,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/pierro...,BROWN
3718,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/hostag...,BLUE
12819,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/a-stor...,BLACK
2353,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/white-...,PURPLE
1649,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/powley...,YELLOW
15130,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/self-p...,BLACK
16313,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/gassed...,BLACK


# Export Picture Data

In [31]:
df.to_csv(data_raw_folder/'pictures.csv',index=False)

# Import Page links

In [5]:
df_pictures = pd.read_csv(data_raw_folder/'pictures.csv')
df_pictures.head()


,image,page,color
0,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/the-ma...,WHITE
1,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/sympho...,WHITE
2,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/the-cr...,WHITE
3,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/the-cr...,WHITE
4,"url(""https://lh3.googleusercontent.com/ci/AC_F...",https://artsandculture.google.com/asset/portra...,WHITE


In [6]:
df_pictures = df_pictures[(df_pictures.image != 'none' )]

In [7]:
len(df_pictures)

16311

In [8]:
links= df_pictures['page'].values.tolist()

In [9]:
def scrapePicturesInfo(picture_link) -> dict:
    scraper = ScrapingPictureInfo(path_driver,picture_link)
    scraper.open()
    time.sleep(2)
    info_dict = scraper.scrape()
    return info_dict

In [10]:
def parallelScrapePictureInfo(link_batch)-> pd.DataFrame:
    with ProcessPoolExecutor(max_workers=8) as executor:
        scrape_info_list = executor.map(scrapePicturesInfo, link_batch)
        scrape_df = pd.DataFrame(list(scrape_info_list))
        return scrape_df

In [11]:
def run_batch(link_batch):
    t1 = time.perf_counter()
    batch_info = parallelScrapePictureInfo(link_batch)
    t2 = time.perf_counter()
    print(f'Finished in {t2-t1} seconds')
    return batch_info


In [12]:
batch_picture_info_1 = run_batch(links[0:1000])

Something went wrong in:  https://artsandculture.google.com/asset/between-red-061-lee-seahyun-and-%EC%9D%B4%EC%84%B8%ED%98%84/6AHePt9WF68hAQ
Something went wrong in:  https://artsandculture.google.com/asset/caixa-de-fazer-amor-lovemaking-box-teresinha-soares/YgFNXA-coHPcnA
Something went wrong in:  Something went wrong in: https://artsandculture.google.com/asset/hougans-visit-wilson-bigaud/bAHQ1XK0iGho0A 
https://artsandculture.google.com/asset/mourning-maria-lassnig/CgEodZktnYgA3w
Something went wrong in:  https://artsandculture.google.com/asset/untitled-rudolf-stingel/EAHYCo-5Xzr04g
Something went wrong in:  https://artsandculture.google.com/asset/retablo-dedicated-by-guadalupe-cortes-de-juarez/-AEAmNwBfXeMgA
Something went wrong in:  https://artsandculture.google.com/asset/quindi-il-calzari-sa-scrivere-curioso-assai-ma-buono-a-sapersi-daniele-milvio/DAEQGt8Z33fmrA
Something went wrong in:  https://artsandculture.google.com/asset/the-old-violin/nQGAOjf3GBDBXQ
Something went wrong in: